In [2]:
import argparse
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
# from text_loader import TextLoader

from tensorflow.contrib.layers import batch_norm

from sklearn import metrics
print(tf.__version__)

1.9.0


In [3]:
data=pd.read_csv('celeba-dataset/newdataset.csv')

In [4]:
data

,image_id,has bags under eyes,is bald,has bangs,has black hair,has blond hair,has brown hair,has bushy eyebrows,is chubby,is wearing eyeglasses,...,has no beard,has oval face,has pale skin,has a pointy nose,has sideburns,is smiling,is wearing a hat,is young,the female,sentence
0,000001.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,1,A opening her mouth slightly woman with no bea...
1,000002.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,A opening her mouth slightly woman with high c...
2,000003.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,A young man with a pointy nose has narrow eyes...
3,000004.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,A woman has a pointy nose and no beard
4,000005.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,A woman is young
5,000006.jpg,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,1,A opening her mouth slightly woman with brown ...
6,000007.jpg,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,A man with bushy eyebrows and black hair has b...
7,000008.jpg,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,A young man with bags under eyes has black hai...
8,000009.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,A young woman with high cheekbones and bangs i...
9,000010.jpg,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,A young woman has high cheekbones and no beard


In [5]:
x=data['sentence']
y=np.array(data.iloc[:,1:26]).tolist()
# y=list(data.iloc[0:,1:2])
# y = [1 if x == '1' else 0 for x in data['is chubby'] ]
y

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [6]:
FLAGS = None
MAX_DOCUMENT_LENGTH = 30

EMBEDDING_SIZE = 20
n_words = 0
MAX_LABEL = 25
WORDS_FEATURE = 'words'
learning_rate = 0.01

sess = tf.Session()


In [7]:

def estimator_spec_for_softmax_classification (logits, labels, mode):

    """Returns EstimatorSpec instance for softmax classification."""

#     predicted_classes = tf.argmax(logits, 1)
    predicted_classes = tf.sigmoid(logits)
    
    print("666")
    print(predicted_classes)
#     print(type(tf.Session().run(predicted_classes)))

    # case 1) inference mode
    
    # ModeKey가 PREDICT이면 inference mode이다. inference는 학습시킨 모델을 새로운 데이터에 적용시키는 것을 말한다.
    if mode == tf.estimator.ModeKeys.PREDICT: 
        
        print("ppp")
        return tf.estimator.EstimatorSpec(mode=mode, predictions={'class': predicted_classes, 'prob': tf.nn.softmax(logits), 'logits': logits})

    
    # case 2) training mode
    
#     onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)
    onehot_labels = labels

    print("777")
    print(onehot_labels)
    print(type(logits))
    
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)    
    
    if mode == tf.estimator.ModeKeys.TRAIN:

        print("ttt")
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op, predictions=logits)

    # case 3) evaluation mode
    
    print("eee!")
    print(labels)
    print(predicted_classes)
    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)}
    print("eee?")
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
def rnn_model(features, labels, mode):
    print(features, labels, mode)
#     anotherlabels = tf.identity(labels, "hihi")
    tf.shape(labels, 'hihi')

    """RNN model to predict from sequence of words to a class."""
    
    # 단어들의 index를 embeddings로 바꾼다. 
    # [n_words(단어 개수), EMBEDDING_SIZE]의 크기를 가지는 matrix를 만들고 순서를 나타내는 단어들의 index를 
    # [batch_size, sequence_length, EMBEDDING_SIZE]에 mapping 한다.
    
    # word_vectors와 word_list를 만든다.
    word_vectors = tf.contrib.layers.embed_sequence(features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)
    word_list = tf.unstack(word_vectors, axis=1)

    
    # embedding size와 같은 hidden size를 가지는 GUR cell을 만든다.
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

    
    # MAX_DOCUMENT_LENGTH의 길이를 가지는 RNN을 만든다. 그리고 각 유닛에 input으로 word_list를 준다.
    # (output, state)쌍이 return된다.
    _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

    ##########중요 여기바꿔야
    # 마지막 유닛의 state 값을 softmax classification의 feature로 넘겨준다.
    logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)
#     print(features[WORDS_FEATURE].eval())
    print("111")
    print(encoding)
    
#     if mode == tf.estimator.ModeKeys.TRAIN:
#         logits = tf.contrib.layers.batch_norm(logits,center=True, scale=True,is_training=True)
#     else:
#         logits = tf.contrib.layers.batch_norm(logits,center=True, scale=True,is_training=False)
    
#     logits=tf.nn.relu(logits, 'relu')
    
    return estimator_spec_for_softmax_classification(logits=logits, labels=labels, mode=mode)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
y

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [11]:
128

128

In [12]:
# train data set, test data set을 나눠준다.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
print(x_train)
print(y_train)
print(x_test)
print(y_test)
# 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)

x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
x_transform_test = vocab_processor.transform(x_test)
x_train = np.array(list(x_transform_train))
x_test = np.array(list(x_transform_test))


n_words = len(vocab_processor.vocabulary_)
print (vocab_processor.vocabulary_)
print('Total words : %d', n_words)

# 모델을 만들어준다.(여기서는 위에서 정의한 rnn_model을 사용한다.)
model_fn = rnn_model
classifier = tf.estimator.Estimator(model_fn=model_fn)


print("111")
# Train
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {WORDS_FEATURE : x_train},
    y = y_train,
    batch_size = 128,
    num_epochs = 1,
    shuffle = True) # shuffle = True : 


# Set up logging for predictions
tensors_to_log = {"hihi": "hihi"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)


['A smiling woman with black hair has no beard and oval face'
 'A opening her mouth slightly woman has no beard'
 'A smiling and young woman has high cheekbones' ...
 'A woman with bangs has a pointy nose'
 'A young man with oval face has sideburns and black hair'
 'A man has no beard']
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]
 ...
 [0 0 1 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
['A man with gray hair and no beard is smiling'
 'A opening hmouth slightly man with black hair has no beard and high cheekbones'
 'A woman has brown hair' ...
 'A wearing eyeglasses woman with high cheekbones and black hair is smiling'
 'A woman has blond hair' 'A woman is young']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]]
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflo

In [13]:
classifier.train(input_fn = train_input_fn, steps = 100, hooks=[logging_hook])


INFO:tensorflow:Calling model_fn.
{'words': <tf.Tensor 'random_shuffle_queue_DequeueUpTo:1' shape=(?, 30) dtype=int64>} Tensor("random_shuffle_queue_DequeueUpTo:2", shape=(?, 25), dtype=int64, device=/device:CPU:0) train
111
Tensor("rnn/gru_cell/add_29:0", shape=(?, 20), dtype=float32)
666
Tensor("Sigmoid:0", shape=(?, 25), dtype=float32)
777
Tensor("random_shuffle_queue_DequeueUpTo:2", shape=(?, 25), dtype=int64, device=/device:CPU:0)
<class 'tensorflow.python.framework.ops.Tensor'>
ttt
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp16lg3fbs/model.ckpt.


InternalError: Blas GEMM launch failed : a.shape=(128, 40), b.shape=(40, 40), m=128, n=40, k=40
	 [[Node: rnn/gru_cell/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](rnn/gru_cell/concat, rnn/gru_cell/gates/kernel/read)]]
	 [[Node: softmax_cross_entropy_loss/value/_157 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1367_softmax_cross_entropy_loss/value", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'rnn/gru_cell/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.la

In [132]:
# train data set, test data set을 나눠준다.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
print(x_train)
print(y_train)
print(x_test)
print(y_test)
# 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)

x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
x_transform_test = vocab_processor.transform(x_test)
x_train = np.array(list(x_transform_train))
x_test = np.array(list(x_transform_test))


n_words = len(vocab_processor.vocabulary_)
print (vocab_processor.vocabulary_)
print('Total words : %d', n_words)

# 모델을 만들어준다.(여기서는 위에서 정의한 rnn_model을 사용한다.)
model_fn = rnn_model
classifier = tf.estimator.Estimator(model_fn=model_fn)


print("111")
# Train
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {WORDS_FEATURE : x_train},
    y = y_train,
    batch_size = 128,
    num_epochs = 1,
    shuffle = True) # shuffle = True : 


# Set up logging for predictions
tensors_to_log = {"hihi": "hihi"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)



classifier.train(input_fn = train_input_fn, steps = 100, hooks=[logging_hook])

# Predict
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {WORDS_FEATURE : x_test},
    y = y_test,
    num_epochs = 1,
    shuffle = False)
predictions = classifier.predict(input_fn = test_input_fn)
y_predicted = np.array(list(p for p in predictions))
print(y_predicted)
# Score using tensorflow
score = classifier.evaluate(input_fn = test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))


In [12]:
def main(unused_argv):
    global n_words
    global vocab_processor
    global classifier

    
    # train data set, test data set을 나눠준다.
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
    print(x_train)
    print(y_train)
    print(x_test)
    print(y_test)
    # 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
    
    x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
    x_transform_test = vocab_processor.transform(x_test)
    x_train = np.array(list(x_transform_train))
    x_test = np.array(list(x_transform_test))
    
    
    n_words = len(vocab_processor.vocabulary_)
    print (vocab_processor.vocabulary_)
    print('Total words : %d', n_words)
    
    # 모델을 만들어준다.(여기서는 위에서 정의한 rnn_model을 사용한다.)
    model_fn = rnn_model
    classifier = tf.estimator.Estimator(model_fn=model_fn)
    
    
    print("111")
    # Train
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {WORDS_FEATURE : x_train},
        y = y_train,
        batch_size = len(x_train),
        num_epochs = None,
        shuffle = True) # shuffle = True : 
    classifier.train(input_fn = train_input_fn, steps = 100)
    
    # Predict
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {WORDS_FEATURE : x_test},
        y = y_test,
        num_epochs = 1,
        shuffle = False)
    predictions = classifier.predict(input_fn = test_input_fn)
    y_predicted = np.array(list(p['class'] for p in predictions))
    
    # Score using tensorflow
    score = classifier.evaluate(input_fn = test_input_fn)
    print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))
    print()

In [13]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--test_with_fake_data',
        default = False,
        help = 'Test the example code with fake data',
        action = 'store_true')
    parser.add_argument(
        '--bow model',
        default = False,
        help = 'Run with BOW model instead of RNN',
        action = 'store_true')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv = [sys.argv[0]] + unparsed)

['A smiling woman with black hair has no beard and oval face'
 'A opening her mouth slightly woman has no beard'
 'A smiling and young woman has high cheekbones' ...
 'A woman with bangs has a pointy nose'
 'A young man with oval face has sideburns and black hair'
 'A man has no beard']
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]
 ...
 [0 0 1 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
['A man with gray hair and no beard is smiling'
 'A opening hmouth slightly man with black hair has no beard and high cheekbones'
 'A woman has brown hair' ...
 'A wearing eyeglasses woman with high cheekbones and black hair is smiling'
 'A woman has blond hair' 'A woman is young']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]]
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflo

ResourceExhaustedError: OOM when allocating tensor with shape[162079,30,20] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: EmbedSequence/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT64, Tparams=DT_FLOAT, _class=["loc:@Adam/Assign"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](EmbedSequence/embeddings/read, random_shuffle_queue_DequeueMany/_147, EmbedSequence/embedding_lookup/axis)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: softmax_cross_entropy_loss/value/_155 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_919_softmax_cross_entropy_loss/value", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want 

In [ ]:
vocab_processor.vocabulary_.reverse(47)

In [ ]:
classifier.predict()

In [ ]:

# train data set, test data set을 나눠준다.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
print(x_train)
print(y_train)
print(x_test)
print(y_test)
# 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)

x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
x_transform_test = vocab_processor.transform(x_test)
x_train = np.array(list(x_transform_train))
x_test = np.array(list(x_transform_test))


In [ ]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {WORDS_FEATURE : x_test},
    y = y_test,
    num_epochs = 1,
    shuffle = False)

# Score using tensorflow
score = classifier.evaluate(input_fn = test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))
print(score)

In [ ]:
for p in predictions:
    print(p)